In [ ]:
import websocket
import json
import time
import threading
import pandas as pd
import numpy as np
import asyncio

# Replace with your actual Deriv API key
API_TOKEN = 'NF0aJKvNJ4IhKnZ'
APP_ID = 64724  # Replace with your app_id if you have one, or use the default 1089

class DerivAPI:
    def __init__(self):
        self.ws = None
        self.data = None
        self.subscribed = False
        self.response = None  # To store proposal responses

    def connect(self, retries=3, delay=5):
        attempt = 0
        while attempt < retries:
            try:
                websocket.enableTrace(True)
                self.ws = websocket.WebSocketApp(
                    f"wss://ws.binaryws.com/websockets/v3?app_id={APP_ID}",
                    on_message=self.on_message,
                    on_error=self.on_error,
                    on_close=self.on_close,
                    on_open=self.on_open
                )
                wst = threading.Thread(target=self.ws.run_forever)
                wst.daemon = True
                wst.start()

                conn_timeout = 10
                while not self.ws.sock or not self.ws.sock.connected and conn_timeout:
                    time.sleep(1)
                    conn_timeout -= 1

                if self.ws.sock and self.ws.sock.connected:
                    print("Connected to WebSocket")
                    return True
            except Exception as e:
                print(f"Connection failed: {e}, retrying in {delay} seconds...")
                time.sleep(delay)
                attempt += 1

        print('Failed to connect to WebSocket after retries')
        return False

    def on_message(self, ws, message):
        data = json.loads(message)
        if 'error' in data:
            print(f"Error: {data['error']['message']}")
        elif 'candles' in data:
            self.data = pd.DataFrame(data['candles'])
            self.data['time'] = pd.to_datetime(self.data['epoch'], unit='s')
            self.subscribed = True
        elif 'proposal' in data:
            self.response = data['proposal']
        print(f"Received data: {data}")

    def on_error(self, ws, error):
        print(f"Error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket connection closed")

    def on_open(self, ws):
        print("WebSocket connection opened")
        self.authorize()

    def authorize(self):
        self.ws.send(json.dumps({"authorize": API_TOKEN}))

    def get_candles(self, symbol, interval, count):
        if not self.ws:
            print("WebSocket is not connected")
            return None

        self.subscribed = False
        self.ws.send(json.dumps({
            "ticks_history": symbol,
            "adjust_start_time": 1,
            "count": count,
            "end": "latest",
            "start": 1,
            "style": "candles",
            "granularity": interval
        }))

        timeout = 10
        while not self.subscribed and timeout:
            time.sleep(1)
            timeout -= 1

        if not self.subscribed:
            print("Failed to receive candle data")
            return None

        return self.data

    async def proposal(self, proposal_params):
        if not self.ws:
            print("WebSocket is not connected")
            return None

        self.response = None
        self.ws.send(json.dumps(proposal_params))

        timeout = 10
        while not self.response and timeout:
            await asyncio.sleep(1)
            timeout -= 1

        if not self.response:
            print("Failed to receive proposal response")
            return None

        return self.response

def calculate_indicators(df):
    df['MA_9'] = df['close'].rolling(window=9).mean()
    df['MA_25'] = df['close'].rolling(window=25).mean()

    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    low_6 = df['low'].rolling(window=14).min()
    high_10 = df['high'].rolling(window=14).max()
    df['%K'] = (df['close'] - low_6) / (high_10 - low_6) * 100
    df['%D'] = df['%K'].rolling(window=3).mean()

    return df

def trading_strategy(df):
    last_row = df.iloc[-1]
    if last_row['MA_9'] > last_row['MA_25'] and last_row['RSI'] > 50 and last_row['%K'] > last_row['%D']:
        return 'BUY'
    elif last_row['MA_9'] < last_row['MA_25'] and last_row['RSI'] < 50 and last_row['%K'] < last_row['%D']:
        return 'SELL'
    else:
        return 'HOLD'

async def execute_trade(api, signal):
    if signal == 'BUY':
        proposal = await api.proposal({
            "proposal": 1, "amount": 100, "barrier": "+0.1", "basis": "payout",
            "contract_type": "CALL", "currency": "USD", "duration": 60, "duration_unit": "s",
            "symbol": "R_100"
        })
    elif signal == 'SELL':
        proposal = await api.proposal({
            "proposal": 1, "amount": 100, "barrier": "-0.1", "basis": "payout",
            "contract_type": "PUT", "currency": "USD", "duration": 60, "duration_unit": "s",
            "symbol": "R_100"
        })
    else:
        return

    if proposal and 'id' in proposal:
        proposal_id = proposal['id']
        api.ws.send(json.dumps({"buy": proposal_id, "price": 100}))
        print(f"Executed {signal} trade with proposal ID: {proposal_id}")
    else:
        print(f"Failed to execute {signal} trade")

async def main():
    deriv_api = DerivAPI()
    if not deriv_api.connect():
        return

    symbol = 'R_100'
    interval = 60
    count = 100

    while True:
        candles = deriv_api.get_candles(symbol, interval, count)
        if candles is not None:
            df = calculate_indicators(candles)
            signal = trading_strategy(df)
            print(f"Current signal: {signal}")

            await execute_trade(deriv_api, signal)
        else:
            print("Failed to retrieve candle data")
        
        time.sleep(60)

if __name__ == "__main__":
    await (main())


--- request header ---
GET /websockets/v3?app_id=64724 HTTP/1.1
Upgrade: websocket
Host: ws.binaryws.com
Origin: https://ws.binaryws.com
Sec-WebSocket-Key: rSojkUEbqRjVdOpqLY/O3Q==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Mon, 11 Nov 2024 15:06:22 GMT
Content-Length: 0
Connection: upgrade
Content-Language: en
Sec-WebSocket-Accept: D28lTmfwOzkeRZJM7OblyqQeyaM=
Upgrade: websocket
CF-Cache-Status: DYNAMIC
Server: cloudflare
CF-RAY: 8e0f2f8f680e65e6-MBA
alt-svc: h3=":443"; ma=86400
-----------------------
Websocket connected
++Sent raw: b'\x81\xa0\xca\x9c\x14\xdc\xb1\xbeu\xa9\xbe\xf4{\xae\xa3\xe6q\xfe\xf0\xbc6\x92\x8c\xacu\x96\x81\xeaZ\x96\xfe\xd5|\x97\xa4\xc66\xa1'
++Sent decoded: fin=1 opcode=1 data=b'{"authorize": "NF0aJKvNJ4IhKnZ"}'


WebSocket connection opened


++Rcv raw: b'\x81~\x05\xc2{"authorize":{"account_list":[{"account_category":"trading","account_type":"binary","broker":"CR","created_at":1710220418,"currency":"USD","currency_type":"fiat","is_disabled":0,"is_virtual":0,"landing_company_name":"svg","linked_to":[],"loginid":"CR6779028"},{"account_category":"trading","account_type":"binary","broker":"CR","created_at":1709901738,"currency":"BTC","currency_type":"crypto","is_disabled":0,"is_virtual":0,"landing_company_name":"svg","linked_to":[],"loginid":"CR6763883"},{"account_category":"trading","account_type":"binary","broker":"CR","created_at":1721128861,"currency":"tUSDT","currency_type":"crypto","is_disabled":0,"is_virtual":0,"landing_company_name":"svg","linked_to":[],"loginid":"CR7371492"},{"account_category":"trading","account_type":"binary","broker":"VRTC","created_at":1709898717,"currency":"USD","currency_type":"fiat","is_disabled":0,"is_virtual":1,"landing_company_name":"virtual","linked_to":[],"loginid":"VRTC10086060"}],"balance

Received data: {'authorize': {'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'broker': 'CR', 'created_at': 1710220418, 'currency': 'USD', 'currency_type': 'fiat', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR6779028'}, {'account_category': 'trading', 'account_type': 'binary', 'broker': 'CR', 'created_at': 1709901738, 'currency': 'BTC', 'currency_type': 'crypto', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR6763883'}, {'account_category': 'trading', 'account_type': 'binary', 'broker': 'CR', 'created_at': 1721128861, 'currency': 'tUSDT', 'currency_type': 'crypto', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR7371492'}, {'account_category': 'trading', 'account_type': 'binary', 'broker': 'VRTC', 'created_at': 1709898717, 'currency': 'USD', 'currency_type': 'fiat', 'is_disabled': 0, 'is_virtual': 1, 'landing_c

++Sent raw: b'\x81\xfe\x00\x84\xb3\xef!\n\xc8\xcdUc\xd0\x84RU\xdb\x86R~\xdc\x9dX(\x89\xcf\x03X\xec\xde\x11:\x91\xc3\x01(\xd2\x8bK\x7f\xc0\x9b~y\xc7\x8eS~\xec\x9bHg\xd6\xcd\x1b*\x82\xc3\x01(\xd0\x80Td\xc7\xcd\x1b*\x82\xdf\x11&\x93\xcdDd\xd7\xcd\x1b*\x91\x83@~\xd6\x9cU(\x9f\xcf\x03y\xc7\x8eS~\x91\xd5\x01;\x9f\xcf\x03y\xc7\x96Mo\x91\xd5\x01(\xd0\x8eOn\xdf\x8aR(\x9f\xcf\x03m\xc1\x8eO\x7f\xdf\x8eSc\xc7\x96\x030\x93\xd9\x11w'
++Sent decoded: fin=1 opcode=1 data=b'{"ticks_history": "R_100", "adjust_start_time": 1, "count": 100, "end": "latest", "start": 1, "style": "candles", "granularity": 60}'


...

++Rcv raw: b'\x81~\x02\x8e{"echo_req":{"amount":100,"barrier":"-0.1","basis":"payout","contract_type":"PUT","currency":"USD","duration":60,"duration_unit":"s","product_type":"basic","proposal":1,"symbol":"R_100"},"msg_type":"proposal","proposal":{"ask_price":48.35,"contract_details":{"barrier":"1021.05"},"date_expiry":1731337706,"date_start":1731337646,"display_value":"48.35","id":"fc3bb338-a39e-decd-d63c-846c8c40e41c","longcode":"Win payout if Volatility 100 Index is strictly lower than entry spot minus 0.10 at 1 minute after contract start time.","payout":100,"spot":1021.15,"spot_time":1731337646,"validation_params":{"payout":{"max":"50000.00"},"stake":{"min":"0.35"}}}}'
++Rcv decoded: fin=1 opcode=1 data=b'{"echo_req":{"amount":100,"barrier":"-0.1","basis":"payout","contract_type":"PUT","currency":"USD","duration":60,"duration_unit":"s","product_type":"basic","proposal":1,"symbol":"R_100"},"msg_type":"proposal","proposal":{"ask_price":48.35,"contract_details":{"barrier":"1021.05"},"

Received data: {'echo_req': {'amount': 100, 'barrier': '-0.1', 'basis': 'payout', 'contract_type': 'PUT', 'currency': 'USD', 'duration': 60, 'duration_unit': 's', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 48.35, 'contract_details': {'barrier': '1021.05'}, 'date_expiry': 1731337706, 'date_start': 1731337646, 'display_value': '48.35', 'id': 'fc3bb338-a39e-decd-d63c-846c8c40e41c', 'longcode': 'Win payout if Volatility 100 Index is strictly lower than entry spot minus 0.10 at 1 minute after contract start time.', 'payout': 100, 'spot': 1021.15, 'spot_time': 1731337646, 'validation_params': {'payout': {'max': '50000.00'}, 'stake': {'min': '0.35'}}}}


CancelledError: 

Connection to remote host was lost. - goodbye


Error: Connection to remote host was lost.
WebSocket connection closed
